In [1]:
#Standard python libraries
import os
import time
import logging
import pickle
 
# Installed libraries
import numpy as np
import pandas as pd
from sklearn.metrics import f1_score
from sklearn.metrics import make_scorer
from sklearn.model_selection import train_test_split
from sklearn import preprocessing

#sklearn
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.experimental import enable_halving_search_cv  # noqa
from sklearn.model_selection import HalvingRandomSearchCV
from sklearn.model_selection import RandomizedSearchCV

In [34]:
df = pd.read_csv(r"C:\Users\kaush\Documents\6390_Kaggle_Project_2\new_train.csv")

In [35]:
df = df.drop(columns=["Unnamed: 0"])
df

,S2_B2_jan,S2_B3_jan,S2_B4_jan,S2_B5_jan,S2_B6_jan,S2_B7_jan,S2_B8_jan,S2_B8A_jan,S2_B9_jan,S2_B11_jan,...,S2_B11_dec,S2_B12_dec,S1_VV_dec,S1_VH_dec,ERA5_temperature_2m_dec,ERA5_total_precipitation_dec,topo_elevation_dec,topo_slope_dec,NDVI_dec,LABELS
0,-14.271277,-21.134172,957.531174,1019.557045,1230.022834,1435.138891,1693.702270,1805.411109,1891.557355,2020.567257,...,2203.025469,795.862978,2747.777894,1688.997611,289.382220,0.000147,1886.384195,0.749163,0.307887,0.0
1,-12.341429,-14.744978,949.227883,895.845779,987.517322,1350.496916,1930.239804,2048.686546,2375.680146,2259.903290,...,2922.382762,1140.796180,2528.590979,1934.261859,266.578370,0.002876,69.876216,1.385904,0.097779,1.0
2,-12.317847,-17.026201,1087.616069,1086.401035,1185.755955,1529.674085,1858.981635,1904.667487,2082.954737,2115.713139,...,2548.726966,521.102434,1556.082048,741.212901,276.467873,0.001622,91.279743,2.125908,0.418492,1.0
3,-15.332478,-20.978203,2987.224569,3163.553102,3412.572434,3908.443371,4044.232162,4231.353770,3682.709329,4518.882323,...,2537.545246,1073.588565,2388.141112,1367.682434,276.101799,0.003720,767.817294,8.427714,0.336528,1.0
4,-11.739502,-11.374262,691.825857,782.111227,478.448214,976.753128,3189.134129,3813.893119,3861.824527,4174.591554,...,4276.302643,371.861994,2080.641023,692.730434,298.663246,0.001763,61.500923,1.203899,0.595404,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59619,-8.163415,-15.237238,987.976575,1151.249683,1243.495362,1570.346203,2985.256589,3424.652420,3504.617647,3786.160943,...,2846.906155,545.811116,4877.934218,2939.084766,293.583787,0.008626,773.400823,3.750506,0.306910,0.0
59620,-11.412780,-18.545065,1961.306816,1904.491667,2287.616313,2794.906732,2868.472367,2952.698071,2972.389632,3199.591485,...,7299.094172,2067.865931,6147.303097,4271.877307,266.000834,0.003052,195.505610,1.661317,0.076089,1.0
59621,-19.757668,-18.974376,1369.005380,1696.975995,2378.074718,2426.659176,2511.191824,2731.803974,2676.593677,2916.115610,...,2695.952941,512.207308,3990.546419,2672.906513,299.309719,0.001287,513.766742,3.598575,0.274659,0.0
59622,-10.536396,-15.386933,3022.282909,3368.075760,3736.875439,4000.780072,4828.072025,5342.635968,5117.677939,5777.358267,...,4433.778527,902.604485,1769.948892,660.408789,292.725187,0.011576,496.085568,12.537847,0.627107,1.0


In [36]:
df.describe()

,S2_B2_jan,S2_B3_jan,S2_B4_jan,S2_B5_jan,S2_B6_jan,S2_B7_jan,S2_B8_jan,S2_B8A_jan,S2_B9_jan,S2_B11_jan,...,S2_B11_dec,S2_B12_dec,S1_VV_dec,S1_VH_dec,ERA5_temperature_2m_dec,ERA5_total_precipitation_dec,topo_elevation_dec,topo_slope_dec,NDVI_dec,LABELS
count,59624.000000,59624.000000,59624.000000,59624.000000,59624.000000,59624.000000,59624.000000,59624.000000,59624.000000,59624.000000,...,59624.000000,59624.000000,59624.000000,59624.000000,59624.000000,59624.000000,59624.000000,59624.000000,59624.000000,59624.000000
mean,-11.795119,-18.078254,2081.046712,2100.501195,2304.266747,2667.026142,3009.954514,3153.526957,3219.598875,3318.671884,...,3354.115868,1292.793264,2070.074244,1295.342055,280.991866,0.003141,502.305559,4.679795,0.259815,0.676976
std,4.208928,4.720095,1934.856503,1933.397044,2085.701681,2339.775198,1972.864966,1919.396478,1926.329600,1891.359026,...,1894.715573,1211.689623,1166.978098,839.315313,14.017163,0.003583,586.159308,6.390935,0.196771,0.467636
min,-57.171004,-74.840663,-56.392936,299.238220,308.116962,136.928851,169.569203,84.967215,88.123196,13.675767,...,190.315943,30.882181,49.438014,47.248169,240.090890,0.000091,-119.033169,-0.163469,-0.643869,0.000000
25%,-13.911653,-20.583776,951.073058,949.720040,992.328940,1251.564736,1782.774981,1885.205452,1950.921249,2040.184965,...,2059.526734,551.741199,1276.246327,748.817994,269.644897,0.000978,124.780914,1.320973,0.100172,0.000000
50%,-11.504172,-17.643541,1163.268277,1210.112302,1437.884700,1648.392700,2302.365800,2523.560200,2597.782990,2761.135720,...,2833.860815,802.781409,1879.542349,1053.972349,278.290841,0.001794,293.217361,2.470939,0.240179,1.000000
75%,-9.322609,-14.926473,2103.385354,2156.902557,2565.968373,2921.320072,3351.444404,3735.071878,3802.460633,4015.688122,...,4084.349790,1457.314397,2650.952606,1646.218963,294.491275,0.003582,692.439658,4.868427,0.401444,1.000000
max,14.057914,7.074079,13624.663133,14274.620618,15724.538745,18868.088113,17166.621988,16868.053089,17286.252694,16836.840939,...,19489.963954,13943.846879,13135.748022,11376.935402,305.419884,0.045823,5376.089584,73.877508,0.796425,1.000000


In [37]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59624 entries, 0 to 59623
Columns: 217 entries, S2_B2_jan to LABELS
dtypes: float64(217)
memory usage: 98.7 MB


In [38]:
df.dtypes

S2_B2_jan                       float64
S2_B3_jan                       float64
S2_B4_jan                       float64
S2_B5_jan                       float64
S2_B6_jan                       float64
                                 ...   
ERA5_total_precipitation_dec    float64
topo_elevation_dec              float64
topo_slope_dec                  float64
NDVI_dec                        float64
LABELS                          float64
Length: 217, dtype: object

In [39]:
labels = df[["LABELS"]]

In [40]:
labels.value_counts()

LABELS
1.0       40364
0.0       19260
dtype: int64

In [41]:
# There is some imbalance in the data. But not much.

In [42]:
df.isna().sum()

S2_B2_jan                       0
S2_B3_jan                       0
S2_B4_jan                       0
S2_B5_jan                       0
S2_B6_jan                       0
                               ..
ERA5_total_precipitation_dec    0
topo_elevation_dec              0
topo_slope_dec                  0
NDVI_dec                        0
LABELS                          0
Length: 217, dtype: int64

In [43]:
df[["LABELS"]] = df[["LABELS"]].astype(int)
df

,S2_B2_jan,S2_B3_jan,S2_B4_jan,S2_B5_jan,S2_B6_jan,S2_B7_jan,S2_B8_jan,S2_B8A_jan,S2_B9_jan,S2_B11_jan,...,S2_B11_dec,S2_B12_dec,S1_VV_dec,S1_VH_dec,ERA5_temperature_2m_dec,ERA5_total_precipitation_dec,topo_elevation_dec,topo_slope_dec,NDVI_dec,LABELS
0,-14.271277,-21.134172,957.531174,1019.557045,1230.022834,1435.138891,1693.702270,1805.411109,1891.557355,2020.567257,...,2203.025469,795.862978,2747.777894,1688.997611,289.382220,0.000147,1886.384195,0.749163,0.307887,0
1,-12.341429,-14.744978,949.227883,895.845779,987.517322,1350.496916,1930.239804,2048.686546,2375.680146,2259.903290,...,2922.382762,1140.796180,2528.590979,1934.261859,266.578370,0.002876,69.876216,1.385904,0.097779,1
2,-12.317847,-17.026201,1087.616069,1086.401035,1185.755955,1529.674085,1858.981635,1904.667487,2082.954737,2115.713139,...,2548.726966,521.102434,1556.082048,741.212901,276.467873,0.001622,91.279743,2.125908,0.418492,1
3,-15.332478,-20.978203,2987.224569,3163.553102,3412.572434,3908.443371,4044.232162,4231.353770,3682.709329,4518.882323,...,2537.545246,1073.588565,2388.141112,1367.682434,276.101799,0.003720,767.817294,8.427714,0.336528,1
4,-11.739502,-11.374262,691.825857,782.111227,478.448214,976.753128,3189.134129,3813.893119,3861.824527,4174.591554,...,4276.302643,371.861994,2080.641023,692.730434,298.663246,0.001763,61.500923,1.203899,0.595404,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59619,-8.163415,-15.237238,987.976575,1151.249683,1243.495362,1570.346203,2985.256589,3424.652420,3504.617647,3786.160943,...,2846.906155,545.811116,4877.934218,2939.084766,293.583787,0.008626,773.400823,3.750506,0.306910,0
59620,-11.412780,-18.545065,1961.306816,1904.491667,2287.616313,2794.906732,2868.472367,2952.698071,2972.389632,3199.591485,...,7299.094172,2067.865931,6147.303097,4271.877307,266.000834,0.003052,195.505610,1.661317,0.076089,1
59621,-19.757668,-18.974376,1369.005380,1696.975995,2378.074718,2426.659176,2511.191824,2731.803974,2676.593677,2916.115610,...,2695.952941,512.207308,3990.546419,2672.906513,299.309719,0.001287,513.766742,3.598575,0.274659,0
59622,-10.536396,-15.386933,3022.282909,3368.075760,3736.875439,4000.780072,4828.072025,5342.635968,5117.677939,5777.358267,...,4433.778527,902.604485,1769.948892,660.408789,292.725187,0.011576,496.085568,12.537847,0.627107,1


In [81]:
X = df.drop(columns=["LABELS"])
X

,S2_B2_jan,S2_B3_jan,S2_B4_jan,S2_B5_jan,S2_B6_jan,S2_B7_jan,S2_B8_jan,S2_B8A_jan,S2_B9_jan,S2_B11_jan,...,S2_B9_dec,S2_B11_dec,S2_B12_dec,S1_VV_dec,S1_VH_dec,ERA5_temperature_2m_dec,ERA5_total_precipitation_dec,topo_elevation_dec,topo_slope_dec,NDVI_dec
0,-14.271277,-21.134172,957.531174,1019.557045,1230.022834,1435.138891,1693.702270,1805.411109,1891.557355,2020.567257,...,1939.707676,2203.025469,795.862978,2747.777894,1688.997611,289.382220,0.000147,1886.384195,0.749163,0.307887
1,-12.341429,-14.744978,949.227883,895.845779,987.517322,1350.496916,1930.239804,2048.686546,2375.680146,2259.903290,...,2816.650882,2922.382762,1140.796180,2528.590979,1934.261859,266.578370,0.002876,69.876216,1.385904,0.097779
2,-12.317847,-17.026201,1087.616069,1086.401035,1185.755955,1529.674085,1858.981635,1904.667487,2082.954737,2115.713139,...,2425.676203,2548.726966,521.102434,1556.082048,741.212901,276.467873,0.001622,91.279743,2.125908,0.418492
3,-15.332478,-20.978203,2987.224569,3163.553102,3412.572434,3908.443371,4044.232162,4231.353770,3682.709329,4518.882323,...,2233.688637,2537.545246,1073.588565,2388.141112,1367.682434,276.101799,0.003720,767.817294,8.427714,0.336528
4,-11.739502,-11.374262,691.825857,782.111227,478.448214,976.753128,3189.134129,3813.893119,3861.824527,4174.591554,...,3569.602117,4276.302643,371.861994,2080.641023,692.730434,298.663246,0.001763,61.500923,1.203899,0.595404
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59619,-8.163415,-15.237238,987.976575,1151.249683,1243.495362,1570.346203,2985.256589,3424.652420,3504.617647,3786.160943,...,2487.672188,2846.906155,545.811116,4877.934218,2939.084766,293.583787,0.008626,773.400823,3.750506,0.306910
59620,-11.412780,-18.545065,1961.306816,1904.491667,2287.616313,2794.906732,2868.472367,2952.698071,2972.389632,3199.591485,...,6940.383810,7299.094172,2067.865931,6147.303097,4271.877307,266.000834,0.003052,195.505610,1.661317,0.076089
59621,-19.757668,-18.974376,1369.005380,1696.975995,2378.074718,2426.659176,2511.191824,2731.803974,2676.593677,2916.115610,...,2364.680153,2695.952941,512.207308,3990.546419,2672.906513,299.309719,0.001287,513.766742,3.598575,0.274659
59622,-10.536396,-15.386933,3022.282909,3368.075760,3736.875439,4000.780072,4828.072025,5342.635968,5117.677939,5777.358267,...,4080.569025,4433.778527,902.604485,1769.948892,660.408789,292.725187,0.011576,496.085568,12.537847,0.627107


In [82]:
y = df[["LABELS"]]

In [74]:
y.to_numpy().ravel()

array([0, 1, 1, ..., 0, 1, 1])

In [46]:
# X=(X-X.mean())/X.std()
# #this will give slightly different results compared to sklearn

# Sklearn feature scaling:

In [83]:
X

,S2_B2_jan,S2_B3_jan,S2_B4_jan,S2_B5_jan,S2_B6_jan,S2_B7_jan,S2_B8_jan,S2_B8A_jan,S2_B9_jan,S2_B11_jan,...,S2_B9_dec,S2_B11_dec,S2_B12_dec,S1_VV_dec,S1_VH_dec,ERA5_temperature_2m_dec,ERA5_total_precipitation_dec,topo_elevation_dec,topo_slope_dec,NDVI_dec
0,-14.271277,-21.134172,957.531174,1019.557045,1230.022834,1435.138891,1693.702270,1805.411109,1891.557355,2020.567257,...,1939.707676,2203.025469,795.862978,2747.777894,1688.997611,289.382220,0.000147,1886.384195,0.749163,0.307887
1,-12.341429,-14.744978,949.227883,895.845779,987.517322,1350.496916,1930.239804,2048.686546,2375.680146,2259.903290,...,2816.650882,2922.382762,1140.796180,2528.590979,1934.261859,266.578370,0.002876,69.876216,1.385904,0.097779
2,-12.317847,-17.026201,1087.616069,1086.401035,1185.755955,1529.674085,1858.981635,1904.667487,2082.954737,2115.713139,...,2425.676203,2548.726966,521.102434,1556.082048,741.212901,276.467873,0.001622,91.279743,2.125908,0.418492
3,-15.332478,-20.978203,2987.224569,3163.553102,3412.572434,3908.443371,4044.232162,4231.353770,3682.709329,4518.882323,...,2233.688637,2537.545246,1073.588565,2388.141112,1367.682434,276.101799,0.003720,767.817294,8.427714,0.336528
4,-11.739502,-11.374262,691.825857,782.111227,478.448214,976.753128,3189.134129,3813.893119,3861.824527,4174.591554,...,3569.602117,4276.302643,371.861994,2080.641023,692.730434,298.663246,0.001763,61.500923,1.203899,0.595404
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59619,-8.163415,-15.237238,987.976575,1151.249683,1243.495362,1570.346203,2985.256589,3424.652420,3504.617647,3786.160943,...,2487.672188,2846.906155,545.811116,4877.934218,2939.084766,293.583787,0.008626,773.400823,3.750506,0.306910
59620,-11.412780,-18.545065,1961.306816,1904.491667,2287.616313,2794.906732,2868.472367,2952.698071,2972.389632,3199.591485,...,6940.383810,7299.094172,2067.865931,6147.303097,4271.877307,266.000834,0.003052,195.505610,1.661317,0.076089
59621,-19.757668,-18.974376,1369.005380,1696.975995,2378.074718,2426.659176,2511.191824,2731.803974,2676.593677,2916.115610,...,2364.680153,2695.952941,512.207308,3990.546419,2672.906513,299.309719,0.001287,513.766742,3.598575,0.274659
59622,-10.536396,-15.386933,3022.282909,3368.075760,3736.875439,4000.780072,4828.072025,5342.635968,5117.677939,5777.358267,...,4080.569025,4433.778527,902.604485,1769.948892,660.408789,292.725187,0.011576,496.085568,12.537847,0.627107


In [84]:
y

,LABELS
0,0
1,1
2,1
3,1
4,1
...,...
59619,0
59620,1
59621,0
59622,1


In [85]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.15, random_state=42)

In [86]:
scaler = preprocessing.StandardScaler()

In [87]:
X_train_scaled = scaler.fit_transform(X_train)

In [88]:
X_test_scaled = scaler.transform(X_test)

# Use X_train_scaled, y_train, X_test_scaled, y_test for evaluation of models

In [91]:
base_rf = RandomForestClassifier().fit(X_train_scaled, y_train.to_numpy().ravel())

In [92]:
base_rf_predictions = base_rf.predict(X_test_scaled)

In [93]:
f1_score(y_test, base_rf_predictions)

0.8872325078143785

# For submitting predictions, we train on the whole dataset:

In [97]:
X

,S2_B2_jan,S2_B3_jan,S2_B4_jan,S2_B5_jan,S2_B6_jan,S2_B7_jan,S2_B8_jan,S2_B8A_jan,S2_B9_jan,S2_B11_jan,...,S2_B9_dec,S2_B11_dec,S2_B12_dec,S1_VV_dec,S1_VH_dec,ERA5_temperature_2m_dec,ERA5_total_precipitation_dec,topo_elevation_dec,topo_slope_dec,NDVI_dec
0,-14.271277,-21.134172,957.531174,1019.557045,1230.022834,1435.138891,1693.702270,1805.411109,1891.557355,2020.567257,...,1939.707676,2203.025469,795.862978,2747.777894,1688.997611,289.382220,0.000147,1886.384195,0.749163,0.307887
1,-12.341429,-14.744978,949.227883,895.845779,987.517322,1350.496916,1930.239804,2048.686546,2375.680146,2259.903290,...,2816.650882,2922.382762,1140.796180,2528.590979,1934.261859,266.578370,0.002876,69.876216,1.385904,0.097779
2,-12.317847,-17.026201,1087.616069,1086.401035,1185.755955,1529.674085,1858.981635,1904.667487,2082.954737,2115.713139,...,2425.676203,2548.726966,521.102434,1556.082048,741.212901,276.467873,0.001622,91.279743,2.125908,0.418492
3,-15.332478,-20.978203,2987.224569,3163.553102,3412.572434,3908.443371,4044.232162,4231.353770,3682.709329,4518.882323,...,2233.688637,2537.545246,1073.588565,2388.141112,1367.682434,276.101799,0.003720,767.817294,8.427714,0.336528
4,-11.739502,-11.374262,691.825857,782.111227,478.448214,976.753128,3189.134129,3813.893119,3861.824527,4174.591554,...,3569.602117,4276.302643,371.861994,2080.641023,692.730434,298.663246,0.001763,61.500923,1.203899,0.595404
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59619,-8.163415,-15.237238,987.976575,1151.249683,1243.495362,1570.346203,2985.256589,3424.652420,3504.617647,3786.160943,...,2487.672188,2846.906155,545.811116,4877.934218,2939.084766,293.583787,0.008626,773.400823,3.750506,0.306910
59620,-11.412780,-18.545065,1961.306816,1904.491667,2287.616313,2794.906732,2868.472367,2952.698071,2972.389632,3199.591485,...,6940.383810,7299.094172,2067.865931,6147.303097,4271.877307,266.000834,0.003052,195.505610,1.661317,0.076089
59621,-19.757668,-18.974376,1369.005380,1696.975995,2378.074718,2426.659176,2511.191824,2731.803974,2676.593677,2916.115610,...,2364.680153,2695.952941,512.207308,3990.546419,2672.906513,299.309719,0.001287,513.766742,3.598575,0.274659
59622,-10.536396,-15.386933,3022.282909,3368.075760,3736.875439,4000.780072,4828.072025,5342.635968,5117.677939,5777.358267,...,4080.569025,4433.778527,902.604485,1769.948892,660.408789,292.725187,0.011576,496.085568,12.537847,0.627107


In [64]:
scaler = preprocessing.StandardScaler()

In [65]:
X_full_scaled = scaler.fit_transform(X)

In [68]:
X_full_scaled

array([[-0.588316  , -0.64743254, -0.58067612, ...,  2.36128681,
        -0.61503746,  0.24430739],
       [-0.12979907,  0.70619419, -0.58496758, ..., -0.73773966,
        -0.51540474, -0.82348094],
       [-0.12419607,  0.22289002, -0.51344323, ..., -0.70122449,
        -0.39961411,  0.80641383],
       ...,
       [-1.89183968, -0.18985412, -0.36801039, ...,  0.01955318,
        -0.16918166,  0.07544061],
       [ 0.29906265,  0.57018854,  0.48646713, ..., -0.01061152,
         1.22957242,  1.86661057],
       [ 0.99853148,  0.00659394, -0.44334937, ..., -0.37417559,
        -0.52505576, -0.04980503]])

In [69]:
# X_test_scaled = scaler.transform(X_test)
test = pd.read_csv(r"C:\Users\kaush\Documents\6390_Kaggle_Project_2\test_nolabels.csv")
test = test.drop(columns=["S.No"])

In [70]:
test

,S2_B2_jan,S2_B3_jan,S2_B4_jan,S2_B5_jan,S2_B6_jan,S2_B7_jan,S2_B8_jan,S2_B8A_jan,S2_B9_jan,S2_B11_jan,...,S2_B9_dec,S2_B11_dec,S2_B12_dec,S1_VV_dec,S1_VH_dec,ERA5_temperature_2m_dec,ERA5_total_precipitation_dec,topo_elevation_dec,topo_slope_dec,NDVI_dec
0,-17.649477,-24.327178,1111.603354,1190.158872,1361.861147,1436.238138,1350.277481,1382.111848,1419.716749,1410.456550,...,777.782928,966.513644,467.731680,1045.355257,633.790963,296.952437,0.000092,276.466775,0.765980,0.000600
1,-14.559686,-19.936119,1764.795592,1856.603435,2156.740323,2633.317506,2731.894209,2878.742338,2621.323845,3201.553256,...,2026.702042,2292.479225,791.909589,2902.059945,1728.924349,298.468523,0.000139,187.130317,2.598537,0.217297
2,-14.208844,-21.404320,1210.320260,1246.028476,1523.531488,1706.652761,2014.364032,2198.544215,2277.422666,2547.302707,...,2103.697056,2369.819453,893.709359,3217.008133,2158.612101,298.045025,0.000104,324.857357,0.758805,0.230539
3,-13.895412,-21.169488,1550.755198,1541.339240,1732.355678,1828.669115,1876.796177,2001.004560,1996.979443,2212.820792,...,2101.697185,2372.614882,841.326953,3259.568699,1979.892417,298.534272,0.000130,184.338553,1.132985,0.214938
4,-11.562658,-13.594914,1171.571568,1143.268311,1296.423152,1376.878830,1488.835033,1530.996415,1667.407479,1737.091382,...,1926.708518,2116.367142,849.233731,2550.935276,1457.042276,298.389238,0.000113,261.577366,0.759891,0.220560
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1195,-14.947773,-18.822142,1190.023326,1293.916708,1634.198686,1820.974390,1991.601005,2146.969823,2272.305089,2424.692034,...,1683.724255,1898.323610,533.950948,2490.286470,1612.946682,298.224866,0.000106,345.330295,2.723475,0.179802
1196,-9.064372,-13.551807,1108.835591,1126.307895,1180.944337,1505.490663,2007.436154,2167.404959,2347.021714,2389.380160,...,1875.711821,2080.958363,948.068460,2299.827937,1361.027977,297.658268,0.000092,235.520899,2.572516,0.257082
1197,-6.723285,-12.445667,1144.816519,1191.156543,1416.713584,1530.773331,1870.857996,1974.730813,1950.921249,2111.789597,...,1501.736041,1685.870938,424.244400,2135.969758,1455.141003,297.323723,0.000092,338.816178,8.861480,0.137137
1198,-9.844837,-14.439588,1027.647855,1018.559373,1012.537732,1283.442883,2019.312516,2181.028384,2323.480860,2578.691039,...,1760.719269,2116.367142,571.508145,1796.549245,835.325927,298.439516,0.000121,248.549132,7.234033,0.297259


In [71]:
X_test_submission= scaler.transform(test)

In [72]:
# #X_train = X_train.to_numpy()
# y_train = y_train.to_numpy().ravel()

# #X_test = X_test.to_numpy()
# y_test = y_test.to_numpy().ravel()

In [75]:
base_rf = RandomForestClassifier().fit(X_full_scaled, y.to_numpy().ravel())

In [77]:
base_rf_predictions = base_rf.predict(X_test_submission)
base_rf_predictions

array([0, 1, 1, ..., 0, 0, 0])

In [76]:
# base_rf_f1 = f1_score(y_test, base_rf_predictions)
# base_rf_f1

In [57]:
# et = ExtraTreesClassifier().fit(X_train,y_train)
# et_predictions = et.predict(X_test)
# et_f1 = f1_score(y_test, et_predictions)
# et_f1

In [78]:
base_rf.get_params()

{'bootstrap': True,
 'ccp_alpha': 0.0,
 'class_weight': None,
 'criterion': 'gini',
 'max_depth': None,
 'max_features': 'auto',
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 100,
 'n_jobs': None,
 'oob_score': False,
 'random_state': None,
 'verbose': 0,
 'warm_start': False}

In [79]:
def numpy_to_submit_df(submit_predictions, filename):
    preds = pd.DataFrame(submit_predictions)
    preds = preds.rename(columns={0: "LABELS"})
    preds.index.names = ["S.No"]
    preds.to_csv("./predictions_submit/"+filename+".csv")

In [80]:
numpy_to_submit_df(base_rf_predictions, "Sklearn_RF_with_different_scaling.csv")

# Grid search starts here:

In [94]:
param_grid_dict = {
    'n_estimators' : [100, 200, 300],
    # 'criterion': ['gini','entropy'],
    # 'max_depth' : [None, 5, 10, 15],
    # 'max_features' : [None, "auto", "log2"],
    # 'class_weight' : [None, "balanced", "balanced_subsample"],
    # 'bootstrap' : [True, False],
    # 'n_jobs': [-1]
}

In [95]:
f1_scorer = make_scorer(f1_score)

In [96]:
rf_clf = RandomForestClassifier()
grid1 = GridSearchCV(estimator = rf_clf, param_grid = param_grid_dict, scoring=f1_scorer, n_jobs=-1, refit = True, cv=3, verbose = 10)

In [103]:
%%time
# fitting the model for grid search 
grid1.fit(X_train_scaled, y_train)

Fitting 3 folds for each of 3 candidates, totalling 9 fits
Wall time: 9min 41s


GridSearchCV(cv=3, estimator=RandomForestClassifier(), n_jobs=-1,
             param_grid={'n_estimators': [100, 200, 300]},
             scoring=make_scorer(f1_score), verbose=10)

In [107]:
grid1.best_estimator_

RandomForestClassifier(n_estimators=300)

In [108]:
grid1.cv_results_

{'mean_fit_time': array([ 80.32110604, 160.96491734, 249.85856207]),
 'std_fit_time': array([0.8381567 , 1.67947163, 1.35880202]),
 'mean_score_time': array([0.65528099, 1.3515017 , 2.70050931]),
 'std_score_time': array([0.04887779, 0.14692907, 0.0738085 ]),
 'param_n_estimators': masked_array(data=[100, 200, 300],
              mask=[False, False, False],
        fill_value='?',
             dtype=object),
 'params': [{'n_estimators': 100},
  {'n_estimators': 200},
  {'n_estimators': 300}],
 'split0_test_score': array([0.88675706, 0.88735632, 0.88854858]),
 'split1_test_score': array([0.88940033, 0.89029733, 0.89186774]),
 'split2_test_score': array([0.88647167, 0.88777607, 0.88833853]),
 'mean_test_score': array([0.88754302, 0.88847657, 0.88958495]),
 'std_test_score': array([0.00131848, 0.00129883, 0.00161645]),
 'rank_test_score': array([3, 2, 1])}

In [109]:
print("Grid1 score is: ",f1_score(y_test, grid1.predict(X_test)))

Grid1 score is:  0.8877355477483232


# Next round of Grid Search:

In [19]:
param_grid_dict = {
    'criterion': ['gini','entropy'],
    # 'max_depth' : [None, 5, 10, 15],
    # 'max_features' : [None, "auto", "log2"],
    # 'class_weight' : [None, "balanced", "balanced_subsample"],
    # 'bootstrap' : [True, False],
    # 'n_jobs': [-1]
}
rf_clf = RandomForestClassifier(n_estimators=300)
grid2 = GridSearchCV(estimator = rf_clf, param_grid = param_grid_dict, scoring=f1_scorer, n_jobs=-1, refit = True, cv=3, verbose = 10)

In [20]:
%%time
# fitting the model for grid search 
grid2.fit(X_train, y_train)

Fitting 3 folds for each of 2 candidates, totalling 6 fits
Wall time: 9min 43s


GridSearchCV(cv=3, estimator=RandomForestClassifier(n_estimators=300),
             n_jobs=-1, param_grid={'criterion': ['gini', 'entropy']},
             scoring=make_scorer(f1_score), verbose=10)

In [37]:
print(grid2.best_estimator_)

RandomForestClassifier(n_estimators=300)


In [22]:
grid2.cv_results_

{'mean_fit_time': array([206.71600095, 267.97504123]),
 'std_fit_time': array([1.05982098, 1.27551041]),
 'mean_score_time': array([1.33791733, 1.49600045]),
 'std_score_time': array([0.16273353, 0.01601857]),
 'param_criterion': masked_array(data=['gini', 'entropy'],
              mask=[False, False],
        fill_value='?',
             dtype=object),
 'params': [{'criterion': 'gini'}, {'criterion': 'entropy'}],
 'split0_test_score': array([0.88871841, 0.88746445]),
 'split1_test_score': array([0.89183786, 0.8922673 ]),
 'split2_test_score': array([0.88888889, 0.88904991]),
 'mean_test_score': array([0.88981505, 0.88959389]),
 'std_test_score': array([0.00143203, 0.00199813]),
 'rank_test_score': array([1, 2])}

In [38]:
#print("Grid1 score is: ",f1_score(y_test, grid1.predict(X_test)))
print("Grid2 score is: ",f1_score(y_test, grid2.predict(X_test)))

Grid2 score is:  0.889741744623011


In [25]:
with open('./models/sklearn_rf_grid2.pkl', 'wb') as files:
    pickle.dump(grid2, files)

# Check whether entropy is still the best criterion - it is!!!

In [37]:
param_grid_dict = {
     'max_depth' : [None, 5, 10, 15],
    # 'max_features' : [None, "auto", "log2"],
    # 'class_weight' : [None, "balanced", "balanced_subsample"],
    # 'bootstrap' : [True, False],
    # 'n_jobs': [-1]
}
rf_clf = RandomForestClassifier(n_estimators=300, criterion='entropy')
grid3 = GridSearchCV(estimator = rf_clf, param_grid = param_grid_dict, scoring=f1_scorer, n_jobs=-1, refit = True, cv=3, verbose = 10)

In [38]:
%%time
# fitting the model for grid search 
grid3.fit(X_train, y_train)

Fitting 3 folds for each of 4 candidates, totalling 12 fits
Wall time: 7min 14s


GridSearchCV(cv=3,
             estimator=RandomForestClassifier(criterion='entropy',
                                              n_estimators=300),
             n_jobs=-1, param_grid={'max_depth': [None, 5, 10, 15]},
             scoring=make_scorer(f1_score), verbose=10)

In [39]:
print(grid3.best_estimator_)

RandomForestClassifier(criterion='entropy', n_estimators=300)


In [40]:
grid3.cv_results_

{'mean_fit_time': array([223.69813148,  94.13127907, 171.56786966, 209.98247727]),
 'std_fit_time': array([0.15309587, 0.63121102, 0.53769367, 0.86472862]),
 'mean_score_time': array([1.02924856, 0.81083171, 0.9288497 , 0.98270623]),
 'std_score_time': array([0.00430929, 0.07350557, 0.03701619, 0.02089362]),
 'param_max_depth': masked_array(data=[None, 5, 10, 15],
              mask=[False, False, False, False],
        fill_value='?',
             dtype=object),
 'params': [{'max_depth': None},
  {'max_depth': 5},
  {'max_depth': 10},
  {'max_depth': 15}],
 'split0_test_score': array([0.88104043, 0.84358227, 0.87063092, 0.878829  ]),
 'split1_test_score': array([0.88727785, 0.84530556, 0.87335646, 0.88520419]),
 'split2_test_score': array([0.88584513, 0.84269188, 0.87185086, 0.8827535 ]),
 'mean_test_score': array([0.88472114, 0.8438599 , 0.87194608, 0.88226223]),
 'std_test_score': array([0.00266757, 0.00108494, 0.00111473, 0.00262574]),
 'rank_test_score': array([1, 4, 3, 2])}

In [41]:
print("Grid1 score is: ",f1_score(y_test, grid1.predict(X_test)))
print("Grid2 score is: ",f1_score(y_test, grid2.predict(X_test)))
print("Grid3 score is: ",f1_score(y_test, grid3.predict(X_test)))#

Grid1 score is:  0.8921676345942552
Grid2 score is:  0.8931978107896795
Grid3 score is:  0.8921200750469044


# Let's make a prediction with grid2:

In [59]:
test = pd.read_csv(r"C:\Users\kaush\Documents\6390_Kaggle_Project_2\test_nolabels.csv")

In [60]:
test = test.drop(columns=["S.No"])

In [42]:
test = (test-test.mean())/test.std()

In [43]:
test

,S2_B2_jan,S2_B3_jan,S2_B4_jan,S2_B5_jan,S2_B6_jan,S2_B7_jan,S2_B8_jan,S2_B8A_jan,S2_B9_jan,S2_B11_jan,...,S2_B9_dec,S2_B11_dec,S2_B12_dec,S1_VV_dec,S1_VH_dec,ERA5_temperature_2m_dec,ERA5_total_precipitation_dec,topo_elevation_dec,topo_slope_dec,NDVI_dec
0,-1.525466,-1.206610,-0.609914,-0.461846,-0.527089,-0.732780,-1.770525,-1.931417,-1.843035,-2.192362,...,-3.504485,-3.605774,-0.971879,-2.597297,-2.066475,-1.517697,-0.530480,0.528054,-0.607787,-3.230098
1,-0.593724,-0.188969,2.166729,1.795263,1.229047,1.711117,1.661690,1.498394,0.698642,1.475342,...,-0.173636,-0.144933,0.412996,-0.048411,-0.186743,0.682086,-0.141871,-0.216358,-0.168723,-0.201562
2,-0.487926,-0.529229,-0.190280,-0.272627,-0.169909,-0.180715,-0.120800,-0.060408,-0.028789,0.135606,...,0.031708,0.056929,0.847881,0.383951,0.550791,0.067606,-0.433551,0.931278,-0.609506,-0.016499
3,-0.393409,-0.474806,1.256869,0.727529,0.291449,0.068387,-0.462546,-0.513108,-0.621991,-0.549327,...,0.026375,0.064225,0.624105,0.442378,0.244029,0.777485,-0.215205,-0.239621,-0.519856,-0.234539
4,0.310045,1.280624,-0.354996,-0.620654,-0.671662,-0.853965,-1.426320,-1.590220,-1.319112,-1.523498,...,-0.440317,-0.604595,0.657882,-0.530435,-0.653412,0.567046,-0.355257,0.403986,-0.609246,-0.155965
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1195,-0.710753,0.069199,-0.276560,-0.110440,0.074590,0.052678,-0.177348,-0.178601,-0.039614,-0.115469,...,-1.088353,-1.173701,-0.688993,-0.613694,-0.385812,0.328549,-0.413453,1.101872,-0.138789,-0.725588
1196,1.063416,1.290614,-0.621679,-0.678096,-0.926791,-0.591398,-0.138010,-0.131770,0.118429,-0.187779,...,-0.576325,-0.697014,1.080100,-0.875155,-0.818215,-0.493564,-0.530353,0.186866,-0.174957,0.354468
1197,1.769383,1.546966,-0.468728,-0.458467,-0.405903,-0.539782,-0.477298,-0.573319,-0.719415,-0.756213,...,-1.573713,-1.728213,-1.157655,-1.100100,-0.656676,-0.978975,-0.530480,1.047592,1.331821,-1.321878
1198,0.828063,1.084869,-0.966799,-1.043017,-1.298853,-1.044719,-0.108507,-0.100549,0.068635,0.199881,...,-0.883008,-0.604595,-0.528550,-1.566057,-1.720552,0.639998,-0.295661,0.295426,0.941900,0.915976


In [44]:
submit_predictions = grid2.predict(test.to_numpy())

In [45]:
def numpy_to_submit_df(submit_predictions, filename):
    preds = pd.DataFrame(submit_predictions)
    preds = preds.rename(columns={0: "LABELS"})
    preds.index.names = ["S.No"]
    preds.to_csv("./predictions_submit/"+filename+".csv")

In [46]:
numpy_to_submit_df(submit_predictions, "Sklearn_RF_new")

In [124]:
submit_predictions_base_rf = base_rf.predict(test.to_numpy())

In [125]:
numpy_to_submit_df(submit_predictions, "Sklearn_basic_RF")

# Fitting a base RF on the entire data:

In [78]:
X.to_numpy().shape

(59624, 216)

In [81]:
y.to_numpy().ravel().shape

(59624,)

In [129]:
base_rf_new_trial = RandomForestClassifier().fit(X_scaled,y.to_numpy().ravel())

In [130]:
submit_predictions_base_rf_entire_data = base_rf_new_trial.predict(test.to_numpy())

In [131]:
numpy_to_submit_df(submit_predictions, "Sklearn_basic_RF_trained_on_full_dataset")

In [ ]:
base_rf.predict(